In [1]:
from pprint import pprint

In [2]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import base64
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']


In [3]:
class Representer():
    def __repr__(self):
        return repr(vars(self))

In [4]:
class Message(Representer):
    def __init__(self, message_id='', thread_id='', label_ids='', date='', from_='',\
                 received='', received_SPF='', subject='', list_unsubscribe='',\
                 size_estimate='', history_id='', internal_date='', parts=list()):
        self.message_id = message_id
        self.thread_id = thread_id
        self.label_ids = label_ids
        self.date = date
        self.from_ = from_
        self.received = received
        self.received_SPF = received_SPF
        self.subject = subject
        self.list_unsubscribe = list_unsubscribe
        self.size_estimate = size_estimate
        self.history_id = history_id
        self.internal_date = internal_date
        # setting this to an empty list instead of parts made it stop adding to a "master" parts list recursively
        self.parts = []
        
    def add(self, part):
        self.parts.append(part)

In [5]:
class Part(Representer):
    def __init__(self, mime_type='', content_type='', content_transfer_encoding='', body_size='', body_data=''):
        self.mime_type = mime_type
        self.content_type = content_type
        self.content_transfer_encoding = content_transfer_encoding
        self.body_size = body_size
        self.body_data = body_data

In [6]:
"""Shows basic usage of the Gmail API.
Lists the user's Gmail labels.
"""
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            '../credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('gmail', 'v1', credentials=creds)



In [39]:
def get_message_list():
    results = service.users().messages().list(userId='me', q='').execute()
    return results

In [40]:
def get_message(msg_id):
    results = service.users().messages().get(userId='me', id=msg_id).execute()
    return results

In [41]:
messages = []

In [42]:
message_threads = get_message_list()['messages']
message_threads[0]

{'id': '17113ceb4d986aab', 'threadId': '17113ceb4d986aab'}

In [22]:
for i, message_thread in enumerate(message_threads):
    print(message_thread)
    message = get_message(message_thread['id'])
    message_id = message['id']
    thread_id = message['threadId']
    label_ids = message['labelIds']
    for header in message['payload']['headers']:
        if header['name'] == 'Date': date = header['value']
        if header['name'] == 'From': from_ = header['value']
        if header['name'] == 'Received': received = header['value']
        if header['name'] == 'Received-SPF': received_SPF = header['value']
        if header['name'] == 'Subject': subject = header['value']
        if header['name'] == 'List-Unsubscribe': list_unsubscribe = header['value']
    size_estimate = message['sizeEstimate']
    history_id = message['historyId']
    internal_date = message['internalDate']
    _message = Message(message_id, thread_id, label_ids, date, from_,\
             received, received_SPF, list_unsubscribe,\
             size_estimate, history_id, internal_date)
    # pprint(message['payload'])
    try:
        for ii, part in enumerate(message['payload']['parts']):
            print(i)
            print(ii)
            mime_type = part['mimeType']
            for header in part['headers']:
                if header['name'] == 'Content-Type': content_type = header['value']
                if header['name'] == 'Content-Transfer-Encoding': content_transfer_encoding = header['value']
            body_size = part['body']['size']
            # body_data = part['body']['data']
            body_data = ''
            _part = Part(mime_type, content_type, content_transfer_encoding, body_size, body_data)
            _message.add(_part)
            print(ii)
            _part = None
    except:
        print(message_id)
        _part = None
    messages.append(_message)
    _message = None

{'id': '17113be362a6f56f', 'threadId': '17113be362a6f56f'}
0
0
0
0
1
1
{'id': '17112b1ce27fcb66', 'threadId': '17112b1ce27fcb66'}
1
0
0
1
1
1
{'id': '171125a020c89b0a', 'threadId': '171125a020c89b0a'}
2
0
0
2
1
1
{'id': '17111c80daba19e3', 'threadId': '17111c80daba19e3'}
3
0
0
3
1
1
{'id': '17111340e74a3e9b', 'threadId': '17111340e74a3e9b'}
4
0
0
4
1
1
{'id': '17110adeff87e35c', 'threadId': '17110adeff87e35c'}
5
0
0
5
1
1
{'id': '171101e651aa226d', 'threadId': '171101e651aa226d'}
171101e651aa226d
{'id': '1710f738c9b989c6', 'threadId': '1710f738c9b989c6'}
7
0
0
7
1
1
{'id': '1710f37bea23792c', 'threadId': '1710f37bea23792c'}
1710f37bea23792c
{'id': '1710f35d099f1ae1', 'threadId': '1710e8e0c39d7669'}
9
0
0
9
1
1
{'id': '1710f16f868f42d0', 'threadId': '1710e8e0c39d7669'}
10
0
0
10
1
1
{'id': '1710ef7c102feb08', 'threadId': '1710ef7c102feb08'}
11
0
0
11
1
1
{'id': '1710eb871fc902a1', 'threadId': '1710eb871fc902a1'}
12
0
0
12
1
1
{'id': '1710e9e2784872e1', 'threadId': '1710e9e2784872e1'}
13

In [43]:
messages = []
for i, message_thread in enumerate(message_threads):
    message_data = {}
    print(message_thread)
    message = get_message(message_thread['id'])
    message_data['message_id'] = message['id']
    message_data['thread_id'] = message['threadId']
    message_data['label_ids'] = message['labelIds']
    for header in message['payload']['headers']:
        if header['name'].lower() == 'date': message_data['date'] = header['value']
        if header['name'].lower() == 'from': message_data['from_'] = header['value']
        if header['name'].lower() == 'received': message_data['received'] = header['value']
        if header['name'].lower() == 'received-spf': message_data['received_SPF'] = header['value']
        if header['name'].lower() == 'subject': message_data['subject'] = header['value']
        if header['name'].lower() == 'list-unsubscribe': message_data['list_unsubscribe'] = header['value']
    message_data['size_estimate'] = message['sizeEstimate']
    message_data['history_id'] = message['historyId']
    message_data['internal_date'] = message['internalDate']
    message_data['parts'] = []
    try:
        for ii, part in enumerate(message['payload']['parts']):
            part_data = {}
            part_data['mime_type'] = part['mimeType']
            for header in part['headers']:
                if header['name'].lower() == 'Content-Type': part_data['content_type'] = header['value']
                if header['name'].lower() == 'Content-Transfer-Encoding': part_data['content_transfer_encoding'] = header['value']
            part_data['body_size'] = part['body']['size']
            # part_data['body_data'] = part['body']['data']
            body_data = ''
            message_data['parts'].append(part_data)
    except:
        print(i, message_thread['id'])
    messages.append(message_data)


{'id': '17113ceb4d986aab', 'threadId': '17113ceb4d986aab'}
{'id': '17113be362a6f56f', 'threadId': '17113be362a6f56f'}
{'id': '17112b1ce27fcb66', 'threadId': '17112b1ce27fcb66'}
{'id': '171125a020c89b0a', 'threadId': '171125a020c89b0a'}
{'id': '17111c80daba19e3', 'threadId': '17111c80daba19e3'}
{'id': '17111340e74a3e9b', 'threadId': '17111340e74a3e9b'}
{'id': '17110adeff87e35c', 'threadId': '17110adeff87e35c'}
{'id': '171101e651aa226d', 'threadId': '171101e651aa226d'}
7 171101e651aa226d
{'id': '1710f738c9b989c6', 'threadId': '1710f738c9b989c6'}
{'id': '1710f37bea23792c', 'threadId': '1710f37bea23792c'}
9 1710f37bea23792c
{'id': '1710f35d099f1ae1', 'threadId': '1710e8e0c39d7669'}
{'id': '1710f16f868f42d0', 'threadId': '1710e8e0c39d7669'}
{'id': '1710ef7c102feb08', 'threadId': '1710ef7c102feb08'}
{'id': '1710eb871fc902a1', 'threadId': '1710eb871fc902a1'}
{'id': '1710e9e2784872e1', 'threadId': '1710e9e2784872e1'}
{'id': '1710e8e0c39d7669', 'threadId': '1710e8e0c39d7669'}
{'id': '1710e78830

In [12]:
for m in messages:
    pprint(vars(m))

{'date': 'Wed, 25 Mar 2020 17:16:14 +0000',
 'from_': 'Twitter <info@twitter.com>',
 'history_id': '1585156574000',
 'internal_date': '',
 'label_ids': ['UNREAD', 'CATEGORY_SOCIAL', 'INBOX'],
 'list_unsubscribe': 100980,
 'message_id': '17112b1ce27fcb66',
 'parts': [{'mime_type': 'text/plain', 'content_type': 'text/plain; charset=UTF-8', 'content_transfer_encoding': 'quoted-printable', 'body_size': 4188, 'body_data': ''},
           {'mime_type': 'text/html', 'content_type': 'text/html; charset=UTF-8', 'content_transfer_encoding': 'quoted-printable', 'body_size': 84518, 'body_data': ''}],
 'received': 'from spruce-goose-ah.twitter.com (spruce-goose-ah.twitter.com. '
             '[199.59.150.77])        by mx.google.com with ESMTPS id '
             'x12si16129823pgi.105.2020.03.25.10.16.14        for '
             '<shortpoet@gmail.com>        (version=TLS1_2 '
             'cipher=ECDHE-RSA-AES128-GCM-SHA256 bits=128/128);        Wed, 25 '
             'Mar 2020 10:16:15 -0700 (PDT)

 'list_unsubscribe': 67706,
 'message_id': '1710394a4cbd4f9a',
 'parts': [{'mime_type': 'text/plain', 'content_type': 'text/plain; charset=UTF-8', 'content_transfer_encoding': 'quoted-printable', 'body_size': 7146, 'body_data': ''},
           {'mime_type': 'text/html', 'content_type': 'text/html; charset=UTF-8', 'content_transfer_encoding': 'quoted-printable', 'body_size': 51082, 'body_data': ''}],
 'received': 'from MTg3MTE3OQ (unknown) by ismtpd0010p1iad1.sendgrid.net (SG) '
             'with HTTP id bNVYiQJlTsC-h0drJHEpXQ Sun, 22 Mar 2020 '
             '18:40:20.306 +0000 (UTC)',
 'received_SPF': 'pass (google.com: domain of '
                 'bounces+1871179-6ddf-shortpoet=gmail.com@email.medium.com '
                 'designates 167.89.65.228 as permitted sender) '
                 'client-ip=167.89.65.228;',
 'size_estimate': '5690504',
 'subject': '<mailto:eb5d3420-23d3-11e2-86db-00259069d5fe@unsub.r.groupon.com?subject=ZWI1ZDM0MjAtMjNkMy0xMWUyLTg2ZGItMDAyNTkwNjlkNWZlOmJhdGN

In [13]:
#_part = None

In [14]:
#_part

In [15]:
#_message = None

In [16]:
#for m in messages:
#    m.parts = None

In [17]:
vars(messages[0])

{'message_id': '17112b1ce27fcb66',
 'thread_id': '17112b1ce27fcb66',
 'label_ids': ['UNREAD', 'CATEGORY_SOCIAL', 'INBOX'],
 'date': 'Wed, 25 Mar 2020 17:16:14 +0000',
 'from_': 'Twitter <info@twitter.com>',
 'received': 'from spruce-goose-ah.twitter.com (spruce-goose-ah.twitter.com. [199.59.150.77])        by mx.google.com with ESMTPS id x12si16129823pgi.105.2020.03.25.10.16.14        for <shortpoet@gmail.com>        (version=TLS1_2 cipher=ECDHE-RSA-AES128-GCM-SHA256 bits=128/128);        Wed, 25 Mar 2020 10:16:15 -0700 (PDT)',
 'received_SPF': 'pass (google.com: domain of n03460b90bd-e8b1b18795dd4786-shortpoet===gmail.com@bounce.twitter.com designates 199.59.150.77 as permitted sender) client-ip=199.59.150.77;',
 'subject': '<https://twitter.com/i/u?t=1&listunsub=t&cn=ZmxleGlibGVfcmVjc18y&sig=fa8f10c87ca6fd093fbd6710d052ab9d02de291d&iid=e8b1b18795dd4786acf50e5c35453e12&uid=762420025&nid=244+26&usbid=9>',
 'list_unsubscribe': 100980,
 'size_estimate': '5695266',
 'history_id': '1585156

In [25]:
pprint(messages)

[{'date': 'Wed, 25 Mar 2020 18:00:00 -0400',
  'from_': 'Eastern Mountain Sports <easternmountainsports@em.ems.com>',
  'history_id': '5695467',
  'internal_date': '1585173600000',
  'label_ids': ['CATEGORY_PROMOTIONS', 'UNREAD', 'INBOX'],
  'list_unsubscribe': '<mailto:a1ei32lf0jagww8y3m2u21xdlc6h9uyf-u@em.ems.com>, '
                      '<http://em.ems.com/public/webform/render_form/default/be9b135894496ff83ad10d00032d4117/unsub/khtpbjcz4fk61gd3fie6xx5mxioqf/cabmjrugurilmsngsranbtphcbwpbjj?td=NPp9ub-jT4SC5YXNOyyaAQD4xxqNzUza-qRwhVszzuQMAtXDVU3S7GSGPsNPigDMxDOL2ZQayxukrOg-QsFBaToXgb0v-wwDFoXf8ei5L-EhauYMmpZSc95Hq4i7F2pSVaVxKRFNOEUvXq16wgzxmiDwZlE1F3iHuz&tid=09606102584529978492965224813883537282801408835736708704358616420757409261405186>',
  'message_id': '17113be362a6f56f',
  'parts': [{'body_size': 875,
             'content_transfer_encoding': 'quoted-printable',
             'content_type': 'text/plain; charset=utf-8',
             'mime_type': 'text/plain'},
            {'body_

              'hf1kpi2bq58u for <shortpoet@gmail.com>; Mon, 23 Mar 2020 '
              '03:36:44 -0400 (envelope-from <Promo@promo.newegg.com>)',
  'received_SPF': 'pass (google.com: domain of promo@promo.newegg.com '
                  'designates 173.213.2.131 as permitted sender) '
                  'client-ip=173.213.2.131;',
  'size_estimate': 34867,
  'subject': 'Savings on WD Black SN750 M.2 500GB SSD',
  'thread_id': '171065c800472c6c'},
 {'date': 'Mon, 23 Mar 2020 02:22:29 +0000',
  'from_': 'Groupon Goods <noreply@r.groupon.com>',
  'history_id': '5691329',
  'internal_date': '1584930149000',
  'label_ids': ['CATEGORY_PROMOTIONS', 'UNREAD', 'INBOX'],
  'list_unsubscribe': '<mailto:eb5d3420-23d3-11e2-86db-00259069d5fe@unsub.r.groupon.com?subject=ZWI1ZDM0MjAtMjNkMy0xMWUyLTg2ZGItMDAyNTkwNjlkNWZlOmJhdGNoOjI4N2QxMDNhLTY2NTgtNDU0Ny1iMDAzLTM2ZTRkMDIxMzg5N18wXzIwMjAwMzIyX3RyZWF0bWVudDA6bnVsbDpVUw==>',
  'message_id': '1710532deebf3790',
  'parts': [],
  'received': 'from mta102s3.r.g

             'mime_type': 'text/plain'},
            {'body_size': 46289,
             'content_transfer_encoding': 'quoted-printable',
             'content_type': 'text/html; charset="utf-8"',
             'mime_type': 'text/html'}],
  'received': 'from localhost (localhost [127.0.0.1]) by mail14.suw15.mcsv.net '
              '(Mailchimp) with ESMTP id 48kbHv4HpVz7061rD for '
              '<shortpoet@gmail.com>; Fri, 20 Mar 2020 20:40:11 +0000 (GMT)',
  'received_SPF': 'pass (google.com: domain of '
                  'bounce-mc.us4_17810271.6424432-deb8e0b0d9@mail14.suw15.mcsv.net '
                  'designates 198.2.182.14 as permitted sender) '
                  'client-ip=198.2.182.14;',
  'size_estimate': 64363,
  'subject': "Cabin Fever? As an Essential Service, we're here for you!\xa0🌲💭",
  'thread_id': '170f9accb8357bc4'},
 {'date': 'Sat, 21 Mar 2020 07:32:58 +1100',
  'from_': 'Serious Eats <newsletter@seriouseats.com>',
  'history_id': '5690428',
  'internal_date': '15847

In [44]:
for i, m in enumerate(messages):
    try:
        if len(m['list_unsubscribe']) < 10:
            print(m)
    except:
        print(i, m['message_id'])        

9 1710f37bea23792c
10 1710f35d099f1ae1
11 1710f16f868f42d0
15 1710e8e0c39d7669
18 1710e3361e3e7719
20 1710ddef6b905652
21 1710caaf028bf11f
25 1710a81582596aba
27 1710a580bdfa076e
28 1710a28684d3268d
29 1710a25520d8c749
30 1710a03f179e6156
34 171095d405bd9749
37 17108a51fc9be488
38 171089c798c277f6
45 171048d30b80f542
50 17103b1586774a9c
51 17103b070e0f3872
53 17103a470f5a8076
55 1710393d23ba5a3b
56 171038bbada38199
66 170fe60536dbd8eb
67 170fe27f22f37136
69 170fdd74940ecd22
71 170fd99b758c5b0f
83 170f97a220ec749c
95 170f5a61326532c5


In [37]:
messages[3]['list_unsubscribe']

KeyError: 'list_unsubscribe'